<a href="https://colab.research.google.com/github/meng-86/Big-Data-Homework---Alexa-can-you-handle-big-data/blob/main/level-2/cloud_etl_Digital_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop2.7.tgz'


Exception: Unable to find py4j in /content/spark-3.2.1-bin-hadoop2.7\python, your SPARK_HOME may not be configured correctly

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)

df.show(5)

# Data cleaning

In [ ]:
# Drop missing values

df = df.dropna()
df.show(5)

In [ ]:
# Count the data frame size
df.count()

In [ ]:
# Print the Schema

df.printSchema()

# Transform Data

In [ ]:
# Data reviewing and table construction 

review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(5)

In [ ]:
# Drop/checking duplications by "review_id"
review_id_df = review_id_df.dropDuplicates(["review_id"])
review_id_df.count()

In [ ]:
# Transform dataframe to fit review_id, print the schema

from pyspark.sql.types import * 

review_id_df = review_id_df.withColumn("customer_id",review_id_df["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_df["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_df["review_date"].cast(DateType()))

review_id_df.printSchema()

In [ ]:
# Transform dataframe to fit customer_id, print the schema
customers_df = df.select(["customer_id"])
customers_df.show(5)

customers_df = customers_df.groupBy("customer_id").count()
customers_df.orderBy("customer_id").select(["customer_id", "count"])
customers_df.show(5)

customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df.show(5)

customers_df = customers_df.withColumn("customer_id",customers_df["customer_id"].cast(IntegerType()))\
.withColumn("customer_count",customers_df["customer_count"].cast(IntegerType()))

customers_df.printSchema()

In [ ]:
# Transform dataframe to product_id, print the schema
products_df = df.select(["product_id", "product_title"])
products_df.count()

# Drop duplications 
products_df = products_df.dropDuplicates(["product_id"])
products_df.count()

# print the schema
products_df.printSchema()

In [ ]:
# Transform dataframe based on "vine", print the schema

vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

vine_df = vine_df.dropDuplicates(["review_id"])

vine_df = vine_df.withColumn("star_rating",vine_df["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine_df["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine_df["total_votes"].cast(IntegerType()))

vine_df.printSchema()

# Write DataFrames to Database

In [12]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cjpyragkbq4i.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", 
          "password": "20181212", 
          "driver":"org.postgresql.Driver"}

In [13]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

NameError: name 'review_id_df' is not defined

In [ ]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)